In [ ]:
import lightgbm as lgbm
import pandas as pd
import numpy as np
from sklearn.metrics import f1_score, accuracy_score
from sklearn.model_selection import train_test_split

params={}
params['data_path']=r'E:\文件夹\大三下\移动互联网课设\大作业\训练\feature\time+fre.csv'

In [ ]:
re_data=pd.read_csv(params['data_path'],keep_default_na=False)##后面这个参数可以解决空值带来的问题
data = np.array(re_data)
data_x = data[:, :-1]
data_y=data[:,-1]#获取文件标签

X_train , X_test, y_train, y_test = train_test_split(  
    data_x,  
    data_y,  
    test_size=0.1,  
    random_state=2020
)  

def modelfit(params , alg ,X_train , X_test, y_train, y_test,early_stopping_rounds=10):
    lgbm_params = params.copy()
    lgbm_params['num_class'] = 4
    lgbm_params.pop('silent');
     
    lgbmtrain = lgbm.Dataset(X_train , y_train , silent=True)
     
    #num_boost_round为弱分类器数目，下面的代码参数里因为已经设置了early_stopping_rounds
    #即性能未提升的次数超过过早停止设置的数值，则停止训练
    cv_result = lgbm.cv(lgbm_params , lgbmtrain , num_boost_round=10000 , nfold=5 , stratified=True , shuffle=True , metrics='multi_logloss' , early_stopping_rounds=early_stopping_rounds , show_stdv=True , seed=0 )
     
    print('best n_estimators:' , len(cv_result['multi_logloss-mean']))
    print('best cv score:' , cv_result['multi_logloss-mean'][-1])
     
    #采用交叉验证得到的最佳参数n_estimators,训练模型
    alg.set_params(n_estimators=len(cv_result['multi_logloss-mean']))
    alg.fit(X_train , y_train)
    
    y_pred = alg.predict(X_test)
    # 模型评价
    f1 = f1_score( y_test, y_pred, average='macro')
    acc = accuracy_score( y_test, y_pred)
    print('f1 : ', f1)
    print('accuracy : ', acc)
    
params = {'boosting_type': 'gbdt',
      'objective': 'multiclass',
      'nthread': -1,
      'silent': True,#是否打印信息，默认False
      'learning_rate': 0.1,
      'num_leaves': 80,
      'max_depth': 5,
      'max_bin': 127,
      'subsample_for_bin': 50000,
      'subsample': 0.8,
      'subsample_freq': 1,
      'colsample_bytree': 0.8,
      'reg_alpha': 1,
      'reg_lambda': 0,
      'min_split_gain': 0.0,
      'min_child_weight': 1,
      'min_child_samples': 20,
      'scale_pos_weight': 1}
 
lgbm1 = lgbm.sklearn.LGBMClassifier(num_class=4 , n_estimators=1000 , seed=0 , **params)
 
modelfit(params , lgbm1 ,X_train , X_test, y_train, y_test)